In [56]:
%matplotlib inline

import os, sys, json, warnings, time
warnings.filterwarnings("ignore")

from pathlib import Path

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn import datasets
from sklearn.metrics import average_precision_score, roc_auc_score, brier_score_loss, precision_recall_curve
from sklearn.calibration import calibration_curve
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

f"Tensor version: {tf.__version__}"

'Tensor version: 2.20.0'

Pfade definieren

In [57]:
# ---------- Paths ----------
if "__file__" in globals():
    ROOT = Path(__file__).resolve().parents[1]
else:
    ROOT = Path.cwd() if Path.cwd().name not in ("notebooks","tools","tests") else Path.cwd().parent

REPORTS_IN  = Path(os.getenv("REPORTS_IN")  or (ROOT / "reports"))         # shared inputs (split, features)
REPORTS_OUT = Path(os.getenv("REPORTS_OUT") or (ROOT / "reports_Charly"))   # your outputs
REPORTS_OUT.mkdir(parents=True, exist_ok=True)

# import loading method
sys.path.insert(0, str(ROOT))
from src.data_loader import load_and_save_data

Speed-Profile erstellen für die Berechnung der Modelle

In [58]:
# ---------- Speed/Profile ----------
SPEED = os.getenv("SPEED", "MEDIUM").upper().strip()
def speed_cfg():
    cfg = dict(CV=5, N_EST=6000, EARLY_STOP=200, MODELS=["lgbm","xgb"], LR=0.03)
    if SPEED == "FAST":
        cfg.update(CV=3, N_EST=2000, EARLY_STOP=50, MODELS=["lgbm"], LR=0.05)
    elif SPEED == "MEDIUM":
        cfg.update(CV=5, N_EST=4000, EARLY_STOP=100)
    elif SPEED == "FULL":
        cfg.update(CV=5, N_EST=8000, EARLY_STOP=300)
    return cfg

CFG        = speed_cfg()
RND        = int(os.getenv("RND", "42"))
CV         = int(os.getenv("CV", str(CFG["CV"])))
N_EST      = int(os.getenv("N_EST", str(CFG["N_EST"])))
ESR        = int(os.getenv("EARLY_STOP", str(CFG["EARLY_STOP"])))
MODELS     = [m.strip() for m in os.getenv("MODELS", ",".join(CFG["MODELS"])).split(",") if m.strip()]
IMB        = os.getenv("IMB", "spw").lower()   # 'iso' (LGBM is_unbalance) or 'spw' (scale_pos_weight)
LR         = float(os.getenv("LR", str(CFG["LR"])))
MEMBER     = os.getenv("MEMBER", "Charlotte")

Load the Dataset

In [82]:
# Laden des Datensatzes und alle -1 durch nan ersetzen, da das ML nicht mit nans umgehen kann
data = load_and_save_data().replace(np.nan, -1)

# zur Kontrolle
total_nans = data.isna().sum().sum()
print("total nans: ",total_nans)


Lade Datensatz aus dem Cache.
total nans:  0


Strukturierung des Datensatzes von Lucas
Es gibt vier verschiedene Modelle, die wir miteinander vergleichen: 
- drop_calc+opt+extras,37
- drop_calc+extras,39 
- drop_calc_only,37
- all_features,57

Entscheidung: alle ps_calc_* droppen (drop_calc_only),missing_count und sum_all_bin hinzufügen (drop_calc+extras),  zusätzlich ps_ind_14 und ps_car_10_cat droppen (drop_calc+opt+extras) 


In [84]:
# ---------- Utils ----------
# Variablen nach Skalenniveau sortieren
def split_cols(cols):
    cat = [c for c in cols if str(c).endswith("_cat")]
    bin = [c for c in cols if str(c).endswith("_bin")]
    num  = [c for c in cols if c not in cat and c not in bin and c != "target"]
    return cat, bin, num

# Extra-variablen erstellen
def fe_extras(X, selected):
    X = X.copy()
    if "missing_count" in selected:
        X["missing_count"] = X.isna().sum(axis=1)
    if "sum_all_bin" in selected:
        b = [c for c in X.columns if str(c).endswith("_bin")]
        X["sum_all_bin"] = X[b].sum(axis=1) if b else 0
    return X

# Datensatz mit ausgewählten Spalten erstellen
def load_selected_feature_list():
    f = REPORTS_IN / "features_selected.csv"
    if not f.exists():
        raise FileNotFoundError(f"Missing {f}. Run feature-gate first.")
    s = pd.read_csv(f)
    if "raw_feature" not in s.columns:
        raise ValueError("features_selected.csv must have column 'raw_feature'.")
    return s["raw_feature"].astype(str).tolist()

var_selected = load_selected_feature_list() # 35 Feature anstatt 57

# erstes Modell mit ausgewhälten Features
X = data[var_selected[:-2]]  # alles von Anfang bis zweitletzte Spalte
y = data["target"]

var_cat, var_bin, var_num = split_cols(X.columns)

X.shape, y.shape


((595212, 35), (595212,))

In [73]:
# zur Kontrolle
print(var_cat) # 13 Variablen
print(var_num) # 11 Variablen
print(var_bin) # 11 Variablen

['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_11_cat']
['ps_ind_01', 'ps_ind_03', 'ps_ind_15', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15']
['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin']


Train-Test Split

In [94]:
## Split von split_indices übernommen: 
split_p = REPORTS_IN / "split_indices.json"
split = json.loads(split_p.read_text())

# Datensatz aufsplitten nach der json-Datei
X_train = X.loc[split["train"]]
y_train = y.loc[split["train"]]
X_test = X.loc[split["test"]]
y_test = y.loc[split["test"]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((200000, 35), (200000,), (50000, 35), (50000,))

### Pre-Processing

Aus der EDA kennen wir die einzelne Variablen. 
- es müssen fehlende Werte ersetzt werden
- es gibt einige kategoriale Variablen mit sehr vielen kategorien -> hier ist ein embedding layer sinnvoller als One-hot encoding
-> ab 10 wird ein Layer eingefügt, für weniger Kategorien wird one-hot-encoding verwendet
- Standardisierung der numerischen Variablen

In [75]:
# Sortieren der kategorialen  Variablen nach vielen und wenigen Kategorien
small_cat = [col for col in var_cat if data[col].nunique() < 10]
large_cat = [col for col in var_cat if data[col].nunique() >= 10]
small_cat,large_cat

(['ps_ind_02_cat',
  'ps_ind_04_cat',
  'ps_ind_05_cat',
  'ps_car_02_cat',
  'ps_car_03_cat',
  'ps_car_05_cat',
  'ps_car_07_cat',
  'ps_car_08_cat',
  'ps_car_09_cat'],
 ['ps_car_01_cat', 'ps_car_04_cat', 'ps_car_06_cat', 'ps_car_11_cat'])

In [95]:
# Pipeline für numerische Features
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Pipeline für kleine kategoriale Features
cat_small_pipeline = Pipeline([
    # kein Imputer, da -1 jetzt als eigene Kategorie mit einbezogen wird
    ("onehot", OneHotEncoder(sparse_output=False, drop='first', handle_unknown="ignore"))
])

# Pipeline für binäre Features
bin_pipeline = Pipeline([
    # bei binären Variablen wird die häufigste Kategorie gewählt, damit sie binär bleiben und keine 3. Kategorie dazu kommt
    ("impute", SimpleImputer(strategy="most_frequent", missing_values=-1))
])

ps_preprocessor = ColumnTransformer([
    ("var_num", num_pipeline, var_num),
    ("small_cat", cat_small_pipeline, small_cat),
    ("var_bin", bin_pipeline, var_bin)
])

ps_preprocessor

,transformers,"[('var_num', ...), ('small_cat', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [96]:
# Transformieren der Daten
X_processed_train = ps_preprocessor.fit_transform(X_train)
X_processed_test = ps_preprocessor.fit_transform(X_test)

# große Kategoriale Variablen
X_large_train = X_train[large_cat]
X_large_test = X_test[large_cat]

print(X_processed_train.shape, X_large_train.shape, y_train.shape, X_processed_test.shape, X_large_test.shape, y_test.shape)

# überprüfen der Datentypen
print(type(X_processed_train), type(X_processed_test))
print(type(X_large_test), type(X_large_train))
print(type(y_train), type(y_test))

# umwandlen von x_large in numpay array, da noch dataframe
X_large_train = X_large_train.to_numpy()
X_large_test = X_large_test.to_numpy()


(200000, 49) (200000, 4) (200000,) (50000, 49) (50000, 4) (50000,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


Model Setup

In [97]:
# Model als Funktion definieren, um Kreuzvalidierung durchführen zu können
def create_model(input_processed, input_large_raw, embedding_dim =8):
    inputs_preprocessed = Input(shape=(input_processed,), name = "Preprocessed_input")
    inputs_large = []
    embeddings = []

    # Embeddings für jede Spalte der großen Variablen bestimmen
    for i, variable in enumerate(input_large_raw):
        inp = Input(shape=(1,), dtype = tf.int32, name =f"cat_input_{i}")
        emb = Flatten()(Embedding(
            input_dim = variable,
            output_dim=embedding_dim
        )(inp))
        inputs_large.append(inp)
        embeddings.append(emb)
    
    # Alles zuEmbeddings und vorbereitete Features zusammenführen
    x = Concatenate() ([inputs_preprocessed] + embeddings)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(1,activation='sigmoid')(x)

    model = Model(inputs = [inputs_preprocessed] + inputs_large, outputs = output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 
    return model

Model Training

warum wird bei der Asugabe keine accuracy mehr angegeben?

In [79]:
# Input-Daten in eine Liste packen
train_inputs = [X_processed_train] + [X_large_train[:, i:i+1] for i in range(X_large_train.shape[1])]
test_inputs  = [X_processed_test]  + [X_large_test[:, i:i+1]  for i in range(X_large_test.shape[1])]

# Modeldefinieren
model_base = create_model(
    X_processed_train.shape[1],
    [int(np.max(X_large_train[:, i])) + 1 for i in range(X_large_train.shape[1])] , 
    embedding_dim=8
    )

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Zeit messen
start = time.perf_counter()

# Modell trainieren
history_base = model_base.fit(
    train_inputs, y_train,
    validation_data=(test_inputs, y_test),
    epochs=50, 
    batch_size=32,
    callbacks=[early_stop]
)

Epoch 1/50
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9630 - loss: 0.1562 - val_accuracy: 0.9634 - val_loss: 0.1530
Epoch 2/50
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9634 - loss: 0.1537 - val_accuracy: 0.9634 - val_loss: 0.1523
Epoch 3/50
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9634 - loss: 0.1530 - val_accuracy: 0.9634 - val_loss: 0.1532
Epoch 4/50
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9634 - loss: 0.1525 - val_accuracy: 0.9634 - val_loss: 0.1530
Epoch 5/50
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9634 - loss: 0.1519 - val_accuracy: 0.9634 - val_loss: 0.1528


In [91]:
print(X_processed_train.shape, X_large_train.shape, y_train.shape, X_processed_test.shape, X_large_test.shape, y_test.shape)

(200000, 49) (200000, 4) (160000,) (50000, 49) (50000, 4) (50000,)


Durchführung der Kreuzvalidierung

In [99]:
#skf = StratifiedKFold(n_splits=CV, shuffle=True, random_state=RND)
skf = StratifiedKFold(n_splits =5, shuffle = True, random_state =RND)

auc_scores = []
ap_scores = []

# Input-Dimensionen vorbereiten
input_processed = X_processed_train.shape[1]
input_large = [int(np.max(X_large_train[:, i])) + 1 for i in range(X_large_train.shape[1])]


for train_idx, val_idx in skf.split(X_processed_train, y_train):
    # der Trainingsdatensatz wird in Trainings- und Validierungsteildatensatz gesplittet:
    X_processed_tr, X_processed_val = X_processed_train[train_idx], X_processed_train[val_idx]
    X_large_tr, X_large_val =  X_large_train[train_idx], X_large_train[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # Inputs für Keras vorbereiten
    train_inputs = [X_processed_tr] + [X_large_tr[:, i:i+1] for i in range(X_large_tr.shape[1])]
    val_inputs =  [X_processed_val] + [X_large_val[:, i:i+1] for i in range(X_large_val.shape[1])]

    # Neues Model pro Fold erstellen
    model = create_model(input_processed, input_large, embedding_dim=8)

    # Early Stopping
    early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    
    # Trainieren der Modelle
    model.fit(
        train_inputs, y_tr,
        validation_data = (val_inputs, y_val),
        epochs = 50, 
        batch_size = 32, 
        callbacks = [early_stop],
        verbose = 0
    )

    # Vorhersagen (Wahrscheinlichkeiten)
    y_val_pred = model.predict(val_inputs, verbose=0).ravel()

    # Metriken
    auc_scores.append(roc_auc_score(y_val, y_val_pred))
    ap_scores.append(average_precision_score(y_val, y_val_pred))

# Ergebnisse
cv_auc_mean = np.mean(auc_scores)
cv_ap_mean = np.mean(ap_scores)

print("CV AUC mean:", cv_auc_mean)
print("CV AP mean:", cv_ap_mean)

CV AUC mean: 0.6228741200088745
CV AP mean: 0.062141322348212104


Vergleichsmetriken berechnen
- Crossvalidation
- Holdout
- Time
- optional: Early_stopping, best_epoche

In [ ]:
# Holdout

y_true = echte Labels (0/1)
y_pred_proba = Modellwahrscheinlichkeiten (z.B. sigmoid-Ausgabe des NN)

hold_auc = roc_auc_score(y_true, y_pred_proba)
hold_ap = average_precision_score(y_true, y_pred_proba)

print("Holdout AUC:", hold_auc)
print("Holdout AP:", hold_ap)

In [ ]:
# Zeitmetriken berechnen
fit_time_s = time.perf_counter() - start

# Vorhersagezeit auf 1000 Samples selbst berechnen, da keine eigene Metrik vorhanden
X_sample = X_test[:1000] # fixes Sample von 1000
start = time.perf_counter()
_ = model.predict(X_sample, verbose=0)
elapsed = (time.perf_counter() - start) * 1000  # in ms
predict_time_ms_per_1k = elapsed

print("Fit time (s):", fit_time_s)
print("Predict time (ms/1k):", predict_time_ms_per_1k)

In [ ]:
# beste Epoche (Epoche mit bestem Wert äquivalent zu best_iteration)
best_iteration = early_stop.stopped_epoch - early_stop.patience

Alter Code für Tensoren, scheinen aber eher hinderlich zu sein

In [ ]:
X_processed_train_t, X_processed_test_t = (
    tf.convert_to_tensor(X_processed_train, dtype=tf.float32),
    tf.convert_to_tensor(X_processed_test, dtype=tf.float32)
)

X_large_train_t, X_large_test_t = (
    tf.convert_to_tensor(X_large_train, dtype=tf.int32),
    tf.convert_to_tensor(X_large_test, dtype=tf.int32)
)

y_train_t, y_test_t = (
    tf.convert_to_tensor(y_train, dtype=tf.int32),
    tf.convert_to_tensor(y_test, dtype=tf.int32)
)

X_processed_train_t.shape, X_large_train_t.shape, y_train_t.shape, X_processed_test_t.shape, X_large_test_t.shape, y_test_t.shape